In [1]:
from importlib import import_module

In [2]:
from jobflow import job, Flow
from jobflow.managers.local import run_locally

In [3]:
@job(data=["x", "y", "z"])
def add_xy(x, y):
    return {"x": x, "y": y, "z": x + y}

In [4]:
@job
def add_xyz(x, y, z):
    return x + y + z

In [5]:
x = 1
y = 2
z = add_xy(x=x, y=y)
w = add_xyz(x=z.output.x, y=z.output.y, z=z.output.z)

In [6]:
flow = Flow([z, w])

In [7]:
def get_function_dict(flow):
    return {
        job.uuid: job.function
        for job in flow.jobs
    }

In [8]:
def get_nodes_dict(function_dict):
    nodes_dict, nodes_mapping_dict = {}, {}
    for i, [k, v] in enumerate(function_dict.items()):
        nodes_dict[i] = v
        nodes_mapping_dict[k] = i
    
    return nodes_dict, nodes_mapping_dict

In [9]:
def get_edges_and_extend_nodes(flow_dict, nodes_mapping_dict, nodes_dict):
    edges_lst = []
    for job in flow_dict['jobs']:
        for k, v in job['function_kwargs'].items():
            if isinstance(v, dict) and '@module' in v and '@class' in v and '@version' in v:
                if len(v['attributes']) == 1:
                    edges_lst.append({'target': nodes_mapping_dict[job["uuid"]], 'targetHandle': k, "source": nodes_mapping_dict[v['uuid']], 'sourceHandle': v['attributes'][0][1]})
                else:
                    edges_lst.append({'target': nodes_mapping_dict[job["uuid"]], 'targetHandle': k, "source": nodes_mapping_dict[v['uuid']], 'sourceHandle': None})
            else:
                if v not in nodes_dict.values():
                    node_index = len(nodes_dict)
                    nodes_dict[node_index] = v
                else:
                    node_index = {tv: tk for tk, tv in nodes_dict.items()}[v]
                edges_lst.append({'target': nodes_mapping_dict[job["uuid"]], 'targetHandle': k, "source": node_index, 'sourceHandle': None})
    return edges_lst, nodes_dict

In [10]:
flow_dict = flow.as_dict()
function_dict = get_function_dict(flow=flow)
nodes_dict, nodes_mapping_dict = get_nodes_dict(function_dict=function_dict)
edges_lst, nodes_dict = get_edges_and_extend_nodes(flow_dict=flow_dict, nodes_mapping_dict=nodes_mapping_dict, nodes_dict=nodes_dict)
edges_lst, nodes_dict

([{'target': 0, 'targetHandle': 'x', 'source': 2, 'sourceHandle': None},
  {'target': 0, 'targetHandle': 'y', 'source': 3, 'sourceHandle': None},
  {'target': 1, 'targetHandle': 'x', 'source': 0, 'sourceHandle': 'x'},
  {'target': 1, 'targetHandle': 'y', 'source': 0, 'sourceHandle': 'y'},
  {'target': 1, 'targetHandle': 'z', 'source': 0, 'sourceHandle': 'z'}],
 {0: <function __main__.add_xy(x, y)>,
  1: <function __main__.add_xyz(x, y, z)>,
  2: 1,
  3: 2})

In [11]:
result = run_locally(flow)
result

2025-01-16 18:02:39,727 INFO Started executing jobs locally
2025-01-16 18:02:41,075 INFO Starting job - add_xy (3832c38e-98dc-44e7-b6d3-b52a5e86d38c)
2025-01-16 18:02:41,076 INFO Finished job - add_xy (3832c38e-98dc-44e7-b6d3-b52a5e86d38c)
2025-01-16 18:02:41,076 INFO Starting job - add_xyz (8ec0e25e-93f6-43cf-9f26-36e88184987e)
2025-01-16 18:02:41,078 INFO Finished job - add_xyz (8ec0e25e-93f6-43cf-9f26-36e88184987e)
2025-01-16 18:02:41,078 INFO Finished executing jobs locally


{'3832c38e-98dc-44e7-b6d3-b52a5e86d38c': {1: Response(output={'x': 1, 'y': 2, 'z': 3}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/janssen/notebooks/2025/2025-01-16-jobflow-qe'))},
 '8ec0e25e-93f6-43cf-9f26-36e88184987e': {1: Response(output=6, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/janssen/notebooks/2025/2025-01-16-jobflow-qe'))}}

In [12]:
for job in flow_dict['jobs']:
    for k, v in job['function_kwargs'].items():
        print(k, v)

x 1
y 2
x {'@module': 'jobflow.core.reference', '@class': 'OutputReference', '@version': None, 'uuid': '3832c38e-98dc-44e7-b6d3-b52a5e86d38c', 'attributes': [['a', 'x']], 'output_schema': None}
y {'@module': 'jobflow.core.reference', '@class': 'OutputReference', '@version': None, 'uuid': '3832c38e-98dc-44e7-b6d3-b52a5e86d38c', 'attributes': [['a', 'y']], 'output_schema': None}
z {'@module': 'jobflow.core.reference', '@class': 'OutputReference', '@version': None, 'uuid': '3832c38e-98dc-44e7-b6d3-b52a5e86d38c', 'attributes': [['a', 'z']], 'output_schema': None}
